FEATURE SELECTION - https://machinelearningmastery.com/feature-selection-machine-learning-python/
    

MODELO CON TRANSACCIONES

AGRUPAMOS POR 5, 15, 30 MINUTOS

NUEVOS CAMPOS 

* DIF_HIGH_LOW
* DIF_OPEN_CLOSE
* NUMERO_TRANSACCIONES_COMPRA
* NUMERO_TRANSACCIONES_VENTA
* CANTIDAD COMPRADA
* CANTIDAD VENDIDA
* SENTIMIENTO




In [11]:
import pandas as pd
import numpy as np
import time, datetime
import math
import warnings
import matplotlib.pyplot as plt
from matplotlib import style

from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import load_boston, load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb

In [12]:
%pylab inline
%matplotlib inline
style.use('ggplot')
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


# CARGA DE DATOS

In [22]:
# FICHERO ALOJADO EN http://dreamlife.es\ripple29Mayo.csv
rutaFicheroTransaccionesXRP = "C:\\Users\\juan.roncero\\Desktop\\Master ML\\Proyecto Final\\Data\\ripple29Mayo.csv"
dfTransacciones = pd.read_csv(rutaFicheroTransaccionesXRP,delimiter=";", decimal=",")


# INDICE

In [21]:
# TRANSFORM
dfTransacciones["FECHA"] = pd.to_datetime(dfTransacciones["FECHA"])
dfTransacciones.index = pd.to_datetime(dfTransacciones["FECHA"], unit='s')

# AGRUPADO

In [18]:
def agrupaDatos(minutos):
    
    df = dfTransacciones
    dfCompras = dfTransacciones[dfTransacciones["TIPOTRANSACCION"]==False]
    dfVentas = dfTransacciones[dfTransacciones["TIPOTRANSACCION"]==True]

    # NUEVAS COLUMNAS
    df["HIGH"] = df["PRICE"]
    df['LOW'] = df["PRICE"]
    df['CANTIDAD_TOTAL'] = df["AMOUNT"]
    dfCompras['NUM_T_COMPRA'] = dfCompras["AMOUNT"]
    dfVentas['NUM_T_VENTA'] = dfVentas["AMOUNT"]

    # MAXIMO, MINIMO, CANTIDAD_TOTAL
    df1 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).agg({'HIGH':np.max, 'LOW':np.min,'CANTIDAD_TOTAL':np.sum})

    # NÚMERO DE TRANSACCIONES DE COMPRA
    df2 = dfCompras.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).count()
    df2 = df2['NUM_T_COMPRA']

    # NÚMERO DE TRANSACCIONES DE VENTA
    df3 = dfVentas.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).count()
    df3 = df3['NUM_T_VENTA']

    # CANTIDAD TOTAL DE MONEDA COMPRADA
    df4 = dfCompras.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).sum() 
    df4["CANTIDAD_COMPRA"] = df4['AMOUNT']
    df4 = df4['CANTIDAD_COMPRA']

    # CANTIDAD TOTAL DE MONEDA VENDIDA
    df5 = dfVentas.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).sum()
    df5["CANTIDAD_VENTA"] = df5['AMOUNT']
    df5 = df5['CANTIDAD_VENTA']

    # TODO - METER ESTO EN EL DF1 Y PROBAR
    #df6 = df.groupby(pd.Grouper(freq=str(minutos)  + 'Min')).agg({'PRIMERO': lambda x: x.head(1),'ULTIMO': lambda x: x.tail(1)})

    
    # OPEN
    df6 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).first()
    df6["OPEN"] = df6["PRICE"]
    df6= df6["OPEN"]

    # CLOSE
    df7 = df.groupby(pd.Grouper(key='FECHA', freq=str(minutos)  + 'Min')).last()
    df7["CLOSE"] = df7["PRICE"]
    df7= df7["CLOSE"]

    df = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=1)

    # NUEVAS COLUMNAS CALCULADAS
    df['VAR_MAX'] = (df['HIGH'] - df['LOW'])/df['LOW'] * 100
    df['VAR_DIA'] = (df['CLOSE'] - df['OPEN'])/df['OPEN'] * 100

    # FILTRAMOS COLUMNAS 
    df = df[['NUM_T_COMPRA','NUM_T_VENTA','CANTIDAD_COMPRA','CANTIDAD_VENTA','VAR_MAX','VAR_DIA','CLOSE']]

    df = df.dropna()
    
    return df

In [19]:
df = agrupaDatos(30)
df.head()

,NUM_T_COMPRA,NUM_T_VENTA,CANTIDAD_COMPRA,CANTIDAD_VENTA,VAR_MAX,VAR_DIA,CLOSE
FECHA,,,,,,,
2018-02-18 01:00:00,36,113,52332.235542,204701.514742,1.206024,-0.173504,1.16797
2018-02-18 01:30:00,203,28,428859.094852,42760.221449,0.850034,0.256849,1.17100
2018-02-18 02:00:00,111,90,105115.316973,117199.070627,0.636752,0.228156,1.17292
2018-02-18 02:30:00,36,92,32214.649095,107239.826570,0.813196,-0.456827,1.17013
2018-02-18 03:00:00,66,141,19158.535265,151231.347337,1.077330,-0.352098,1.16601


MODELOS 

In [29]:
def probarAlgoritmos(X_ent, X_test, y_ent, y_test):
    
    # TODO
    # 
    # 1. CONTROL SOBREAJUSTE
    # 2. MEDIR CON TODAS LAS MÉTRICAS POSIBLES
    # 3. CROSS VALIDATION...
    # OTROS ALGORITMOS
    
    resumen = pd.DataFrame(columns=('ALGORITMO','SCORE','PARÁMETROS'))
    
    # REGRESIÓN LINEAL
    rl = LinearRegression()
    rl.fit(X_ent,y_ent)
    scoreRL = rl.score(X_test,y_test)    
 
    new_row = ['REGRESION LINEAL',scoreRL,'']
    resumen.loc[len(resumen)] = new_row 
    
    del rl
    
    # K-VECINOS
    mejorNumVecinos =0
    mejorScoreKNN = 0

    for i in range(1,10):
        knn = KNeighborsRegressor(n_neighbors=i)
        knn.fit(X_ent, y_ent)
        score = knn.score(X_test,y_test)
        if(score >mejorScoreKNN): 
            mejorScoreKNN = score
            mejorNumVecinos = i
            
    new_row = ['K-VECINOS',mejorScoreKNN,'n_neighbors:%s' %mejorNumVecinos]
    resumen.loc[len(resumen)] = new_row 
        
    del knn
    
    # RIDGE
    ridge = Ridge()
    ridge.fit(X_ent, y_ent)
    score = ridge.score(X_test, y_test)
    
    new_row = ['RIDGE',score,'']
    resumen.loc[len(resumen)] = new_row 
    
    del ridge
    
    # ARBOL DE DECISION
    mejorProfundidad = 0
    mejorScoreTree = 0
    for i in range(1,6):
        arbol = DecisionTreeRegressor(max_depth=i)
        arbol.fit(X_ent,y_ent)
        mejorScoreTree = arbol.score(X_test,y_test)
        
        if(score>mejorScoreTree):
            mejorScoreTree = score
            mejorProfundidad = i        
    
    new_row = ['ÁRBOL DE DECISIÓN',mejorScoreTree,'max_depth:%s' %mejorProfundidad]
    resumen.loc[len(resumen)] = new_row 
    
    del arbol
    
    # SVM SVC
    clf = svm.SVC()
    clf.fit(X_ent,y_ent.astype('int'))
    score = clf.score(X_test,y_test.astype('int'))
    
    new_row = ['SVM SVC',score,'']
    resumen.loc[len(resumen)] = new_row  
    
    del clf
    
    
    # SVM SVR
    clf = svm.SVR(kernel='poly')
    clf.fit(X_ent,y_ent.astype('int'))
    score = clf.score(X_test,y_test.astype('int'))
    
    new_row = ['SVM SVR poly',score,'']
    resumen.loc[len(resumen)] = new_row  
    
    del clf
    
    
    
    
    #Random forest
    score = 0
    clf = RandomForestClassifier(n_jobs=2, random_state=0)
    clf.fit(X_ent,y_ent.astype('int'))
    score = clf.score(X_test,y_test.astype('int'))
    
    new_row = ['RANDOM FOREST',score,'']
    resumen.loc[len(resumen)] = new_row  
    
    # XGboost
    mejorProfundidad = 0
    mejorScoreXGBoost = 0
    for profundidad in range(2,6):
        gbm = xgb.XGBClassifier(max_depth=profundidad, n_estimators=300, learning_rate=0.05)
        gbm.fit(X_ent,y_ent)
        score = gbm.score(X_test,y_test)
        
        if(score>mejorScoreXGBoost):
            mejorScoreXGBoost = score
            mejorProfundidad = profundidad 
            new_row = ['XGBOOST',mejorScoreXGBoost,'max_depth:%s' %mejorProfundidad]
            resumen.loc[len(resumen)] = new_row  
        
    return resumen
    

In [31]:
#iris = load_iris()
#X_ent, X_test, y_ent, y_test = train_test_split(iris.data, iris.target)
#probarAlgoritmos(X_ent, X_test, y_ent, y_test)